In [1]:
# Install kaggle package

!pip install kaggle

In [2]:
from google.colab import files

In [3]:
# Upload Kaggle API token (kaggle.json)

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"dasol109","key":"522d25066231f0a28008aa7a67923f90"}'}

In [4]:
# Move token and give permission

!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

In [5]:
# Download data

!kaggle datasets download -d kunalgupta2616/flickr-8k-images-with-captions

 99% 1.03G/1.04G [00:09<00:00, 92.0MB/s]
100% 1.04G/1.04G [00:10<00:00, 111MB/s] 


In [6]:
# Unzip data

!unzip -q flickr-8k-images-with-captions.zip

In [19]:
import os
import pickle
import numpy as np
import nltk
import torch
import torchvision.transforms as transforms
import torch.utils.data as data
import torch.nn as nn
import matplotlib.pyplot as plt

from multiprocessing import cpu_count
from collections import Counter
from PIL import Image
from pycocotools.coco import COCO
from torch.nn.utils.rnn import pack_padded_sequence
from torchvision.models import resnet152

In [8]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [9]:
def resize_image(image_dir, output_dir, size):
  print("hellO")
  if not os.path.exists(output_dir):
    os.makedirs(output_dir)

  images = os.listdir(image_dir)
  num_image = len(images)
  for i, image in enumerate(images):
    with open(os.path.join(image_dir, image), 'r+b') as f:
      with Image.open(f) as img:
        img = img.resize(size, Image.ANTIALIAS)
        img.save(os.path.join(output_dir, image), img.format)
    if (i + 1) % 1000 == 0:
      print("[{}/{}] Image Resized".format(i+1, num_image))


In [10]:
image_dir = '/content/Images'
output_dir = '/content/train'

image_size = [256, 256]

resize_image(image_dir, output_dir, image_size)

hellO
[1000/8091] Image Resized
[2000/8091] Image Resized
[3000/8091] Image Resized
[4000/8091] Image Resized
[5000/8091] Image Resized
[6000/8091] Image Resized
[7000/8091] Image Resized
[8000/8091] Image Resized


In [11]:
class Vocabulary(object):
  def __init__(self):
    self.word2idx = {}
    self.idx2word = {}
    self.idx = 0

  def add_word(self, word):
    if not word in self.word2idx:
      self.word2idx[word] = self.idx
      self.idx2word[self.idx] = word
      self.idx += 1
    
  def __call__(self, word):
    if not word in self.word2idx:
      return self.word2idx['<unk>']
    return self.word2idx[word]

  def __len__(self):
    return len(self.word2idx)

In [12]:
import pandas as pd

image_tokens=pd.read_csv("/content/captions.txt",sep=',',names=["id","caption"])
token1 = image_tokens.drop([image_tokens.index[0]])
token1.head()

,id,caption
1,1000268201_693b08cb0e.jpg,A child in a pink dress is climbing up a set o...
2,1000268201_693b08cb0e.jpg,A girl going into a wooden building .
3,1000268201_693b08cb0e.jpg,A little girl climbing into a wooden playhouse .
4,1000268201_693b08cb0e.jpg,A little girl climbing the stairs to her playh...
5,1000268201_693b08cb0e.jpg,A little girl in a pink dress going into a woo...


In [ ]:
print(token1.index)

Int64Index([    1,     2,     3,     4,     5,     6,     7,     8,     9,
               10,
            ...
            40446, 40447, 40448, 40449, 40450, 40451, 40452, 40453, 40454,
            40455],
           dtype='int64', length=40455)


In [13]:
def bulid_vocab(token, threshold):
  dataset = token
  counter = Counter()

  for i, value in dataset.iterrows():
    
    caption = str(value["caption"])
    tokens = nltk.tokenize.word_tokenize(caption.lower())
    counter.update(tokens)

    print(caption)
    #print(counter.items())

  words = [word for word, cnt in counter.items() if cnt >=threshold]

  vocab = Vocabulary()
  vocab.add_word('<pad>')
  vocab.add_word('<start>')
  vocab.add_word('<end>')
  vocab.add_word('<unk>')

  for w in words:
    vocab.add_word(w)

  return vocab

In [14]:
vocab = bulid_vocab(token1, 4)

with open('/content/vocab.pkl', 'wb') as f:
  pickle.dump(vocab, f)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
A dog is jumping over a log in a wooded area while carrying another log .
A dog with a stick in his mouth jumps over a fallen tree in the forest .
Dog carries stick and jumps over a log .
The dog carries a stick and jumps over a log in the woods .
The dog jumps over the log with a stick in its mouth .
A black and brown dog is running between two cement barriers with snow .
a dog runs down the cold aisle .
Black and tan small dog walking with perked ears .
The black and brown dog walks toward the camera , in an enclosed , snowy area .
The brown and black dog is running through a snowy street .
A furry dog is running through a doorway leading to plants .
A furry , tan dog is outside on a patio .
A short , furry dog stands on a brick floor in front of a group of potted plants .
A very hairy dog is running down a hall .
Flowers are behind the fluffy dog that is coming up the step .
A brown and black dog runs through the leaves .
A brown dog is running .


In [102]:
class DS(data.Dataset):
  def __init__(self, root, df, vocab, transform=None):
    self.root = root
    self.df = df
    self.vocab = vocab
    self.transform = transform
    
  def __getitem__(self, index):
    key = self.df.loc[index]
    caption = self.df.loc[index, ["caption"]]
    id = self.df.loc[index,["id"]]
    path = self.df.loc[index,["id"]]

    image = Image.open(os.path.join(self.root, path)).convert("RGB")
    if self.trainsform is not None:
      image = self.trainsform(image)

    tokens = nltk.tokenize.word_tokenize(str(caption).lower())
    caption = []
    caption.append(self.vocab('<start>'))
    caption.extend([self.vocab(token) for token in tokens])
    caption.append(self.ocab('<end>'))
    target = torch.Tensor(caption)
    return image, target

  def __len__(self):
    return len(self.df)

In [103]:
token1.to_json('/content/json.json', orient = 'records')
print(len(token1))

40455


In [104]:
with open('/content/vocab.pkl', 'rb') as f:
  vocab = pickle.load(f)

transform = transforms.Compose([
              transforms.RandomCrop(224),
              transforms.RandomHorizontalFlip(),
              transforms.ToTensor(),
              transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])
json = token1.to_json(orient = 'records')
train_dataset = DS('/content/train', token1, vocab = vocab, transform = transform)
token1.head()

,id,caption
1,1000268201_693b08cb0e.jpg,A child in a pink dress is climbing up a set o...
2,1000268201_693b08cb0e.jpg,A girl going into a wooden building .
3,1000268201_693b08cb0e.jpg,A little girl climbing into a wooden playhouse .
4,1000268201_693b08cb0e.jpg,A little girl climbing the stairs to her playh...
5,1000268201_693b08cb0e.jpg,A little girl in a pink dress going into a woo...


In [105]:
def collate_fn(data):

  data.sort(key=lambda x: len(x[1]), reverse=True)
  images, captions = zip(*data)

  images = torch.stack(images, 0)
  lengths = [len(cap) for cap in captions]
  targets = torch.zero(len(captions), max(lengths)).long()
  for i, cap in enumerate(captions):
    end = lengths[i]
    target[i, :end] = cap[:end]

  return images, targets, lengths

In [106]:
num_workers = 0

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 128, shuffle = True, num_workers= num_workers, collate_fn = collate_fn)

In [107]:
class EncoderCNN(nn.Module):
  def __init__(self):
    super().__init__()
    pretrained = resnet152(pretrained = True)
    layers = list(pretrained.children())[:-1]
    self.resnet = nn.Sequential(*layers)
    self.linear = nn.Linear(pretrained.fc.in_features, 256)
    self.bn = nn.BatchNorm1d(256, momentum=0.01)

  def forward(self, x):
    with torch.no_grad():
      features = self.resnet(x)
    features = features.view(features.size(0), -1)
    latent_vector = self.bn(self.linear(features))
    return latent_vector

In [108]:
class DecoderLSTM(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    self.embed = nn.Embedding(vocab_size, 256)
    self.lstm = nn.LSTM(256, 512, 1, batch_first=True)
    self.linear = nn.Linear(512, vocab_size)

  def forward(self, features, captions, lengths):
    embeddings = self.embed(caption)
    embeddings = torch.cat((features.unsqueeze(1), embeddings), 1)
    packed = pack_padded_sequence(embeddings, lengths, batch_first = True)
    hiddens, _ = self.lstm(packed)
    outputs = self.linear(hiddens[0])
    return outputs


In [109]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

encoder = EncoderCNN().to(device)
decoder = DecoderLSTM(len(vocab)).to(device)

criterion = nn.CrossEntropyLoss()
params = list(decoder.parameters()) + list(encoder.linear.parameters()) + list(encoder.bn.parameters())
optimizer = torch.optim.Adam(params, lr=0.001)

In [114]:
EPOCHS = 3

total_step = len(train_loader)

encoder.train()
decoder.train()

for epoch in range(EPOCHS):
  for i, (images, captions, lengths) in enumerate(train_loader):
    images, captions = imgaes.to(device), captions.tp(device)
    targets = pack_padded_sequence(captions, length, batch_first = True)[0]
    optimizer.zero_grad()

    features = encoder(images)
    outputs = decoder(features, captions, lengths)
    loss = criterion(outputs, targets)
    loss.backward()
    optimizer.step()

    if (i+1) % 100 == 0:
      print("EPOCH: [{}/{}], STEP: [{}/{}], LOSS: {:.4f}".format(epoch, EPOCHS, i + 1, total_step, loss.item()))

TypeError: ignored

In [ ]:
def load_image(image_path):
  img = Image.open(image_path).convert('RGB')
  img = img.resize([224, 224], Image.LANCZOS)
  transform = transforms.Compose([
                  transforms.ToTensor(),
                  transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
  ])
  img = transform(img).unsqueexe(0)

  
  return img

In [ ]:
image_path = ''
img = load_image(image_path).to(device)

In [ ]:
max_sequence_length = 20

encoder.eval()
decoder.eval()

word_ids= []

with torch.no_grad():
  inputs = encoder(img).unsqueeze(1)
  states = None
  for i in range(max_sequence_length):
    hiddens, states = decoder.lstm(inputs, states)
    outputs = decoder.linear(hidden.squeeze(1))
    _, predicted = outputs.max(1)
    word_ids.append(predicted)
    inputs = decoder.embed(predicted).unsqueeze(1)

  word_ids = torch.stack(word_ids, 1)
  word_ids = word_ids[0].detach().cpu().numpy()

  caption = []

  for id in word_ids:
    word = vocab.idx2word[id]
    caption.appenf(word)
    if word == '<end>':
      break
  caption = ' '.join(caption)


In [ ]:
print(caption)
image = Image.open(image_path)
plt.imshow(np.asarray(image)